<img src="./logo_UNSAM.jpg" align="right" width="150" /> 

# TS9: Filtrado No Lineal

**Alumnas:** María Victoria Poric y Agustina Rocío Paolini Rosso  
**Carrera:** Ingeniería Biomédica    
**Materia:** Análisis y Procesamiento de Señales    
**Fecha:** Noviembre 2025 

## Consigna
En esta tarea semanal se analizarán técnicas no lineales de estimación/sustracción de la señal de movimiento de base $b(n)$.

La señal filtrada se estima como:

$$
\hat{x} = s - \hat{b}
$$

donde $s$ es la señal de ECG registrada con interferencias y $\hat{x}$ la estimación del ECG sin interferencias.

Se pide implementar las siguientes estimaciones de $b$:

1) Filtro de Mediana: se estima $\hat{b}$ mediante:

$$
\hat{b} = \text{med}_{600}(\text{med}_{200}(s))
$$

donde $\text{med}_t(s)$ es el operador de mediana aplicado sobre una ventana de $t$ milisegundos. Es decir:

$$
\hat{b}[n] = \text{med}(m[n], m[n-1], \ldots, m[n-600 \cdot fs])
$$

con:

$$
m[n] = \text{med}(s[n], s[n-1], \ldots, s[n-200 \cdot fs])
$$

2) Interpolación mediante Splines Cúbicos: a partir de las detecciones provistas en `qrs_detections`, se puede estimar el nivel isoeléctrico (segmento PQ).  

Para cada detección $n_i$, se anticipa un tiempo $n_0$ para evitar incluir las ondas P y Q. Se generan así los puntos:

$$
S = \{ (m_i, s(m_i)) \mid i = 1, 2, \ldots, Q \}
$$

donde:

$$
m_i = n_i - n_0
$$

Luego se estima la línea de base mediante un spline cúbico:

$$
\hat{b}(n) = \text{spline}_3(S, n),\quad n = 1,\ldots,N
$$

La línea de base debe estar muestreada a la misma frecuencia que el ECG.



3) Filtro Adaptado (Matched Filter): utilice el patrón provisto en `qrs_pattern1` para implementar un filtro adaptado y diseñar un detector de latidos.

a) Explicación conceptual: explique cómo se utiliza la salida del filtro adaptado para detectar latidos, discutiendo ventajas y limitaciones.

b) Detección y Métricas: compare las detecciones obtenidas con `qrs_detections`. Calcule métricas como sensibilidad (Recall) y valor predictivo positivo (Precisión).


*Bonus*:

- Proponga alguna mejora para los estimadores de la línea de base $b$, por ejemplo mediante técnicas multirate.
- Discuta si el detector basado en filtro adaptado funcionaría con un patrón perteneciente a otro paciente o registro.


## Introducción
La señal electrocardiográfica (ECG) constituye una de las herramientas fundamentales en el análisis de la actividad eléctrica del corazón. Sin embargo, su correcta interpretación depende en gran medida de la calidad del registro. Entre las interferencias más comunes y problemáticas se encuentra el movimiento de línea de base, una oscilación de muy baja frecuencia (típicamente menor a 0.5 Hz) producida por fenómenos como la respiración, desplazamientos del electrodo y variaciones en la impedancia piel-electrodo. Aunque esta interferencia es de baja frecuencia, puede poseer una energía considerable, generando desplazamientos lentos que distorsionan la forma del ECG y dificultan tareas posteriores como la detección del complejo QRS, la medición del segmento ST o el análisis de la morfología de las ondas P y T.

Si se modela el registro como la suma de la actividad cardíaca real y el movimiento de línea de base, puede escribirse:
$$
s(n) = x(n) + b(n)
$$

donde $s(n)$ representa el ECG registrado, $x(n)$ la señal cardíaca deseada y $b(n)$ la interferencia de baja frecuencia. El objetivo consiste entonces en obtener una estimación $\hat{b}(n)$ de esta última para recuperar:
$$
\hat{x}(n) = s(n) - \hat{b}(n)
$$

A diferencia de los filtros lineales tradicionales explorados previamente en la TS8, en este trabajo se abordan métodos no lineales de estimación, capaces de ofrecer mayor robustez frente a artefactos impulsivos, componentes no estacionarios y variaciones abruptas en la línea de base. Estas técnicas permiten modelar el movimiento de forma más flexible y extraerlo sin distorsionar las características esenciales del ECG, particularmente las pendientes abruptas del complejo QRS.

En este contexto se estudian dos estrategias principales. La primera es el filtro de mediana aplicado en forma secuencial sobre ventanas equivalentes a `200 ms` y `600 ms`. Este método explota la naturaleza no lineal de la mediana para suprimir componentes de alta frecuencia sin afectar estructuras más anchas, lo que lo convierte en un estimador eficaz del desplazamiento de base. La segunda técnica se basa en interpolación mediante splines cúbicos, utilizando puntos del segmento PQ (región isoeléctrica del ciclo cardíaco) para reconstruir una versión suave de la línea de base que respete la frecuencia de muestreo original. Ambas estrategias apuntan a preservar la integridad del complejo QRS y de las ondas más rápidas, evitando la distorsión asociada a filtros lineales agresivos.

Finalmente, se incorpora el estudio del filtro adaptado (matched filter), una herramienta clásica de detección óptima en presencia de ruido blanco. Mediante la correlación del ECG con un patrón QRS representativo, este filtro realza la ocurrencia de latidos y permite evaluar métricas de desempeño como sensibilidad y valor predictivo positivo. Su análisis permite además reflexionar sobre la dependencia del detector respecto del patrón utilizado y su capacidad de generalización entre distintos pacientes.

En conjunto, estas técnicas permiten comprender mejor las fortalezas y limitaciones de los métodos no lineales aplicados a la corrección de la línea de base y la detección de latidos, aspectos esenciales para el procesamiento robusto de señales biomédicas en contextos reales.

## Análisis y desarrollo
En primer lugar, se inició mediante la estimación de la línea de base. Para ello, se elaboró un código que comienza con la carga del archivo `ECG_TP4.mat` que contiene la señal de ECG, las detecciones QRS provistas y un patrón típico del complejo QRS. Se fijó la frecuencia de muestreo en `fs = 1000 Hz`, lo cual resulta fundamental ya que todas las ventanas temporales se traducen directamente en tamaños de kernel para los distintos métodos de estimación. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sig
from scipy.interpolate import CubicSpline
import scipy.io as sio

# ============================================================
#  CONFIGURACIÓN INICIAL
fs = 1000  # Hz

#  LECTURA DE ECG
mat_struct = sio.loadmat("./ECG_TP4.mat")
ecg_one_lead = mat_struct["ecg_lead"].flatten()
qrs_det = mat_struct["qrs_detections"].flatten()
cant_muestras = len(ecg_one_lead)

FileNotFoundError: [Errno 2] No such file or directory: './ECG_TP4.mat'

El primer método implementado es la sustracción de la línea de base utilizando dos filtros de mediana aplicados en cascada. El procedimiento sigue exactamente la consigna:
$$
\hat{b} = \text{med}_{600}(\text{med}_{200}(s))
$$
Dado que el operador de mediana requiere una ventana impar, las ventanas de `200 ms` y `600 ms` se convierten en `201` y `601` muestras. 

El primer filtro suaviza oscilaciones rápidas, preservando transiciones abruptas como el QRS. El segundo extrae únicamente la componente de muy baja frecuencia asociada a la línea de base. De esta manera, restando a la señal original esta estimación realizada, se obtiene la señal de ECG corregida. 

In [ ]:
#  1) FILTRO DE MEDIANA

kernel_med1 = 201 
kernel_med2 = 601 

# Aplicamos el filtro de mediana de 200 ms
med_200 = sig.medfilt(ecg_one_lead, kernel_size=kernel_med1)

# Luego el filtro de 600 ms sobre la salida anterior
baseline_med = sig.medfilt(med_200, kernel_size=kernel_med2)

# ECG filtrado
ecg_med_corr = ecg_one_lead - baseline_med

plt.figure(figsize=(12,4))
plt.plot(ecg_one_lead, label="ECG original", alpha=0.6)
plt.plot(baseline_med, 'k', linewidth=2, label="Baseline estimada (medianas)")
plt.plot(ecg_med_corr, 'g', label="ECG corregido")
plt.legend()
plt.title("Método 1: Filtro de mediana")
plt.show()